# 2020 Election Data - Virginia

## Purpose:
* Scrape Virginia Department of Election for 2020 voter preferences by locality

## Dependencies

In [77]:
# Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import numpy as np

## For Multiple Outputs 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
## Assign website url
vde_url = "https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/Index.html"

In [5]:
## Connect to url
vde_page = requests.get(vde_url)

## Confirm connection
vde_page.status_code

200

In [6]:
## Parse website
vde_page_parsed = BeautifulSoup(vde_page.content, "html.parser")

## Check parsed website
vde_page_parsed

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, maximum-scale=1, minimum-scale=1" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="en-us" http-equiv="Content-Language"/>
<meta content="en-US" name="language"/>
<meta content="max-age=0" http-equiv="cache-control">
<meta content="no-cache" http-equiv="cache-control">
<meta content="0" http-equiv="expires">
<meta content="Tue, 01 Jan 1980 1:00:00 GMT" http-equiv="expires"/>
<meta content="no-cache" http-equiv="pragma"/>
<!--[if IE 7]>
<html class="ie ie7 no-js" lang="en-US">
<![endif]-->
<!--[if IE 8]>
<html class="ie ie8 no-js" lang="en-US">
<style type="text/css">

   .header_container { 
       background-color:#FFFFFF;       
    } 
  .stickymenu {
 background-color:#FFFFFF; 
border-bottom:2px solid #CCCCCC;
}

    </style>
<![endif]-->
<!--[if !(I

In [26]:
## Check some "a" tags
vde_tag = vde_page_parsed.find_all("a")

vde_tag


[<a accesskey="c" href="#skipPoint" tabindex="1" title="Skip to Content">Skip to Content</a>,
 <a class="virginiaLogo" href="https://www.virginia.gov/" tabindex="2" title="Virginia.gov portal"><span class="hidden-alt-text">Virginia.gov portal</span></a>,
 <a href="https://www.virginia.gov/government/state-employees/agency-directory" tabindex="3" title="Virginia agency websites">Agencies</a>,
 <a href="https://www.governor.virginia.gov" tabindex="4" title="Virginia Governor">Governor</a>,
 <a accesskey="s" href="https://www.virginia.gov/search" tabindex="5" title="Search Virginia.Gov">Search Virginia.Gov</a>,
 <a href="https://www.elections.virginia.gov/">Home</a>,
 <a href="https://www.elections.virginia.gov/registration/">Registration</a>,
 <a href="https://www.elections.virginia.gov/registration/how-to-register/">How to Register</a>,
 <a href="https://www.elections.virginia.gov/registration/view-your-info/">View Your Info</a>,
 <a href="https://www.elections.virginia.gov/registration

In [29]:
## Extract relevant links
links = set()
for vde_tag in vde_page_parsed.find_all("a"):
    if vde_tag.attrs.get("href") is not None:
        href = vde_tag.attrs.get("href")
        ## Limit hyper links to 'COUNTY' or 'CITY'
        if ("COUNTY" in href or "CITY" in href) and "https:" not in href:
            links.update(["https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/" + href[2:]])

## Check links
links

{'https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/ACCOMACK_COUNTY/Index.html',
 'https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/ALBEMARLE_COUNTY/Index.html',
 'https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/ALEXANDRIA_CITY/Index.html',
 'https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/ALLEGHANY_COUNTY/Index.html',
 'https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/AMELIA_COUNTY/Index.html',
 'https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/AMHERST_COUNTY/Index.html',
 'https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/APPOMATTOX_COUNTY/Index.html',
 'https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/ARLINGTON_COUNTY/Index.html',
 'https://results.elec

In [31]:
test_url = list(links)[0]

In [32]:
## Connect to url
page = requests.get(test_url)

## Parse website
soup = BeautifulSoup(page.content, 'html.parser')

soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, maximum-scale=1, minimum-scale=1" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="en-us" http-equiv="Content-Language"/>
<meta content="en-US" name="language"/>
<meta content="max-age=0" http-equiv="cache-control">
<meta content="no-cache" http-equiv="cache-control">
<meta content="0" http-equiv="expires">
<meta content="Tue, 01 Jan 1980 1:00:00 GMT" http-equiv="expires"/>
<meta content="no-cache" http-equiv="pragma"/>
<!--[if IE 7]>
<html class="ie ie7 no-js" lang="en-US">
<![endif]-->
<!--[if IE 8]>
<html class="ie ie8 no-js" lang="en-US">
<style type="text/css">

   .header_container { 
       background-color:#FFFFFF;       
    } 
  .stickymenu {
 background-color:#FFFFFF; 
border-bottom:2px solid #CCCCCC;
}

    </style>
<![endif]-->
<!--[if !(I

In [78]:
## Get table information
content = [tr.get_text().split('\n') for tr in soup.find_all("tr")]

## Check content
content[2][1]    
## Check content
content[2][6]

'Donald J. Trump\r'

'53.54%'

In [63]:
def vde_scraper(url):
    '''
    Goal: Obtain all voter percentage of a candidate per locality

    Arguments: Website url (string)

    Returns: Election information per locality from websites (list of lists)
    '''
    ## Connect to url
    page = requests.get(url)

    ## Parse website
    soup = BeautifulSoup(page.content, 'html.parser')

    ## Get country name
    locality = url.replace("https://results.elections.virginia.gov/vaelections/2020%20November%20General/Site/Locality/","")
    locality = locality.replace("/Index.html","")
    locality = locality.replace("_"," ")
    
    ## Get table information
    content = [tr.get_text().split('\n') for tr in soup.find_all("tr")]

    ## Check if there is no "tr" elements first
    if content is not None:
        ## Create empty list to populate
        vde_content = []
        ## Loop through each object of content
        for i in range(len(content)):
            ## Remove table headers
            if content[i][1] != "Candidate" and content[i][1] != "Response":
                ## Append locality name to table
                content[i].append(locality)
                ## Append table information to empty list
                vde_content.append(content[i])

        ## Return list of list
        return vde_content

In [64]:
vde_scraper(test_url)

[['',
  'Joseph R. Biden\r',
  '                                                \t\t\t\t\t\t',
  'Democratic',
  '',
  '4,501',
  '45.31%',
  '',
  'WESTMORELAND COUNTY'],
 ['',
  'Donald J. Trump\r',
  '                                                \t\t\t\t\t\t',
  'Republican',
  '',
  '5,318',
  '53.54%',
  '',
  'WESTMORELAND COUNTY'],
 ['',
  'Jo Jorgensen\r',
  '                                                \t\t\t\t\t\t',
  'Libertarian',
  '',
  '90',
  '0.91%',
  '',
  'WESTMORELAND COUNTY'],
 ['',
  'Write In\r',
  '                                                \t\t\t\t\t\t',
  'Write-In',
  '',
  '24',
  '0.24%',
  '',
  'WESTMORELAND COUNTY'],
 ['',
  'Mark R. Warner\r',
  '                                                \t\t\t\t\t\t',
  'Democratic',
  '',
  '4,821',
  '49.02%',
  '',
  'WESTMORELAND COUNTY'],
 ['',
  'Daniel M. Gade\r',
  '                                                \t\t\t\t\t\t',
  'Republican',
  '',
  '5,009',
  '50.94%',
  '',
  'WESTMORELAND

In [66]:
def link_scrape(urls, sleep=3):
    '''
    Goal: Apply vde_scraper function to all urls

    Arguments: Set or List containing website urls

    Returns: Data frame containing each locality page's voter preferences
    '''
    ## Create empty list
    scraped_data = []

    ## Loop through each url
    for url in urls:
        ## Apply function...
        scrape = vde_scraper(url)
        ## If function does not return None...
        if scrape is not None:
            ## Add to empty list
            scraped_data.extend(vde_scraper(url))
        ## Sleep
        time.sleep(random.uniform(0,sleep))
    ## Convert list of lists to data frame
    df = pd.DataFrame(scraped_data)

    return df

## Store data scraper
vde0 = link_scrape(links)

In [67]:
vde0

0                   1                                                  2  \
0        Joseph R. Biden\r                                                ...   
1        Donald J. Trump\r                                                ...   
2           Jo Jorgensen\r                                                ...   
3               Write In\r                                                ...   
4         Mark R. Warner\r                                                ...   
...  ..                ...                                                ...   
2860            Write In\r                                                ...   
2861                 Yes\r                                                ...   
2862                  No\r                                                ...   
2863                 Yes\r                                                ...   
2864                  No\r                                                ...   

                3 4       5       6 7                     8  
0      Democratic     4,501  45.31%     WESTMORELAND COUNTY  
1      Republican     5,318  53.54%     WESTMORELAND COUNTY  
2     Libertarian        90   0.91%     WESTMORELAND COUNTY  
3        Write-In        24   0.24%     WESTMORELAND COUNTY  
4      Democratic     4,821  49.02%     WESTMORELAND COUNTY  
...           ... ..    ...     ... ..                  ...  
2860                     18   0.70%             NORTON CITY  
2861                  1,042  69.61%             NORTON CITY  
2862                    455  30.39%             NORTON CITY  
2863                  1,417  91.36%             NORTON CITY  
2864                    134   8.64%             NORTON CITY  

[2865 rows x 9 columns]

In [72]:
vde1 = vde0.rename(columns={0:"empty1",
                1:"Candidate",
                2:"empty2",
                3:"Affiliation",
                4:"empty3",
                5:"Count",
                6:"Percentage",
                7:"empty4",
                8:"Locality"})

vde1

empty1          Candidate  \
0            Joseph R. Biden\r   
1            Donald J. Trump\r   
2               Jo Jorgensen\r   
3                   Write In\r   
4             Mark R. Warner\r   
...     ...                ...   
2860                Write In\r   
2861                     Yes\r   
2862                      No\r   
2863                     Yes\r   
2864                      No\r   

                                                 empty2  Affiliation empty3  \
0                                                   ...   Democratic          
1                                                   ...   Republican          
2                                                   ...  Libertarian          
3                                                   ...     Write-In          
4                                                   ...   Democratic          
...                                                 ...          ...    ...   
2860                                                ...                       
2861                                                ...                       
2862                                                ...                       
2863                                                ...                       
2864                                                ...                       

      Count Percentage empty4             Locality  
0     4,501     45.31%         WESTMORELAND COUNTY  
1     5,318     53.54%         WESTMORELAND COUNTY  
2        90      0.91%         WESTMORELAND COUNTY  
3        24      0.24%         WESTMORELAND COUNTY  
4     4,821     49.02%         WESTMORELAND COUNTY  
...     ...        ...    ...                  ...  
2860     18      0.70%                 NORTON CITY  
2861  1,042     69.61%                 NORTON CITY  
2862    455     30.39%                 NORTON CITY  
2863  1,417     91.36%                 NORTON CITY  
2864    134      8.64%                 NORTON CITY  

[2865 rows x 9 columns]

In [79]:
vde1['empty1'].unique()
vde1['empty2'].unique()
vde1['empty3'].unique()
vde1['empty4'].unique()

array([''], dtype=object)

array(['                                                \t\t\t\t\t\t'],
      dtype=object)

array([''], dtype=object)

array([''], dtype=object)

In [123]:
## Remove empty columns
vde2 = vde1.drop(columns= ['empty1','empty2','empty3','empty4'])

vde2

Candidate  Affiliation  Count Percentage             Locality
0     Joseph R. Biden\r   Democratic  4,501     45.31%  WESTMORELAND COUNTY
1     Donald J. Trump\r   Republican  5,318     53.54%  WESTMORELAND COUNTY
2        Jo Jorgensen\r  Libertarian     90      0.91%  WESTMORELAND COUNTY
3            Write In\r     Write-In     24      0.24%  WESTMORELAND COUNTY
4      Mark R. Warner\r   Democratic  4,821     49.02%  WESTMORELAND COUNTY
...                 ...          ...    ...        ...                  ...
2860         Write In\r                  18      0.70%          NORTON CITY
2861              Yes\r               1,042     69.61%          NORTON CITY
2862               No\r                 455     30.39%          NORTON CITY
2863              Yes\r               1,417     91.36%          NORTON CITY
2864               No\r                 134      8.64%          NORTON CITY

[2865 rows x 5 columns]

In [124]:
## Remove extra characters in candidate value
vde2['Candidate'] = vde2['Candidate'].str.replace("\r","")

## Check value
vde2['Candidate'].unique()



array(['Joseph R. Biden', 'Donald J. Trump', 'Jo Jorgensen', 'Write In',
       'Mark R. Warner', 'Daniel M. Gade', 'Qasim Rashid',
       'Robert J. Wittman', 'Michael J. Fitzpatrick', 'Robin Schick',
       'Thomas M. Moncure Jr.', 'Richard M. "Mike" Cabrey',
       'Wayne DiRosario', 'Alan Darlington', 'Caryn Self Sullivan',
       'Dallas W. Leamon', 'Joseph "Joey" Paul King',
       'Kathryn S. Wittman', 'Bobby D. Greene', 'Terry A. Cosgrove',
       'Audra Lucas-Peyton', 'Michelle "Shelly" Payne',
       'Theresa "Terri" McClure', 'Brent T. Steffey', 'Yes', 'No',
       'Jennifer T. Wexton', 'Aliscia N. Andrews', 'April F. Wilkerson',
       'Mark J. Griffin', 'Matthew E. "Matt" Bass', 'B. Cameron Webb',
       'Robert G. "Bob" Good', 'Michael E. Mattox',
       'James A. "Champ" Nowlin', 'J. Scott Lowman',
       'James H. Higginbotham II', 'Tracy A. Emerson',
       'Joseph B. "Benny" David ', 'Richard T. Adams Jr.',
       'Scott V. Fisher', 'Roger L. Vance', 'C. B. "Kelly" Bu

In [125]:
## Convert percenmtage to float
vde2['percent'] = vde2['Percentage'].str.rstrip('%').astype('float') / 100.0

vde2['percent'].dtypes

dtype('float64')

In [126]:
## Create dummy for Democratic majority in Presidential election
vde2["P_Democrat"] = np.where(((vde2['Candidate'] == "Joseph R. Biden") & (vde2['percent'] > 0.5)), 1, 0)

## Check
vde2.loc[vde2['P_Democrat'] == 1,]

Candidate Affiliation    Count Percentage               Locality  \
79    Joseph R. Biden  Democratic  138,372     61.54%         LOUDOUN COUNTY   
146   Joseph R. Biden  Democratic    3,667     54.47%     NORTHAMPTON COUNTY   
225   Joseph R. Biden  Democratic   11,022     64.51%      HARRISONBURG CITY   
296   Joseph R. Biden  Democratic   11,710     60.40%          DANVILLE CITY   
336   Joseph R. Biden  Democratic    4,552     57.27%       BRUNSWICK COUNTY   
370   Joseph R. Biden  Democratic    1,612     67.70%           EMPORIA CITY   
419   Joseph R. Biden  Democratic    7,146     81.03%      FALLS CHURCH CITY   
465   Joseph R. Biden  Democratic   66,240     80.28%        ALEXANDRIA CITY   
479   Joseph R. Biden  Democratic    6,610     54.60%        WINCHESTER CITY   
535   Joseph R. Biden  Democratic    9,174     68.04%           FAIRFAX CITY   
549   Joseph R. Biden  Democratic  116,572     63.65%         HENRICO COUNTY   
600   Joseph R. Biden  Democratic    6,981     53.74%          STAUNTON CITY   
682   Joseph R. Biden  Democratic   25,553     51.50%      JAMES CITY COUNTY   
780   Joseph R. Biden  Democratic    4,973     51.94%   PRINCE EDWARD COUNTY   
794   Joseph R. Biden  Democratic    4,790     69.59%      WILLIAMSBURG CITY   
890   Joseph R. Biden  Democratic   66,377     52.22%        CHESAPEAKE CITY   
1024  Joseph R. Biden  Democratic   20,696     85.50%   CHARLOTTESVILLE CITY   
1125  Joseph R. Biden  Democratic    2,624     59.09%    CHARLES CITY COUNTY   
1188  Joseph R. Biden  Democratic    1,791     64.84%         LEXINGTON CITY   
1208  Joseph R. Biden  Democratic   12,389     87.75%        PETERSBURG CITY   
1240  Joseph R. Biden  Democratic   28,676     57.77%           SUFFOLK CITY   
1318  Joseph R. Biden  Democratic  106,935     52.45%    CHESTERFIELD COUNTY   
1382  Joseph R. Biden  Democratic   10,356     61.03%          MANASSAS CITY   
1411  Joseph R. Biden  Democratic   42,466     65.68%       ALBEMARLE COUNTY   
1564  Joseph R. Biden  Democratic    3,766     62.63%      MARTINSVILLE CITY   
1717  Joseph R. Biden  Democratic   40,245     50.54%        STAFFORD COUNTY   
1763  Joseph R. Biden  Democratic    2,827     55.56%          SUSSEX COUNTY   
1956  Joseph R. Biden  Democratic   46,220     70.14%           HAMPTON CITY   
2039  Joseph R. Biden  Democratic   92,175     82.92%          RICHMOND CITY   
2131  Joseph R. Biden  Democratic   53,099     65.39%      NEWPORT NEWS CITY   
2200  Joseph R. Biden  Democratic   64,440     71.69%           NORFOLK CITY   
2234  Joseph R. Biden  Democratic   30,948     69.42%        PORTSMOUTH CITY   
2295  Joseph R. Biden  Democratic    2,525     62.22%          FRANKLIN CITY   
2309  Joseph R. Biden  Democratic   26,773     61.80%           ROANOKE CITY   
2408  Joseph R. Biden  Democratic   23,218     51.55%      MONTGOMERY COUNTY   
2421  Joseph R. Biden  Democratic  117,393     51.59%    VIRGINIA BEACH CITY   
2556  Joseph R. Biden  Democratic  142,863     62.64%  PRINCE WILLIAM COUNTY   
2580  Joseph R. Biden  Democratic    2,397     53.61%           SURRY COUNTY   
2625  Joseph R. Biden  Democratic    3,358     53.13%           RADFORD CITY   
2655  Joseph R. Biden  Democratic    5,430     56.52%          HOPEWELL CITY   
2680  Joseph R. Biden  Democratic    2,627     57.43%     GREENSVILLE COUNTY   
2694  Joseph R. Biden  Democratic  105,344     80.60%       ARLINGTON COUNTY   
2741  Joseph R. Biden  Democratic    8,517     66.22%    FREDERICKSBURG CITY   
2783  Joseph R. Biden  Democratic  419,943     69.89%         FAIRFAX COUNTY   
2823  Joseph R. Biden  Democratic    3,992     65.58%     MANASSAS PARK CITY   

      percent  P_Democrat  
79     0.6154           1  
146    0.5447           1  
225    0.6451           1  
296    0.6040           1  
336    0.5727           1  
370    0.6770           1  
419    0.8103           1  
465    0.8028           1  
479    0.5460           1  
535    0.6804           1  
549    0.6365          

In [127]:
## Create dummy for Republican majority in Presidential election
vde2["P_Republican"] = np.where(((vde2['Candidate'] == "Donald J. Trump") & (vde2['percent'] > 0.5)), 1, 0)

## Check
vde2.loc[vde2['P_Republican'] == 1,]

Candidate Affiliation   Count Percentage             Locality  \
1     Donald J. Trump  Republican   5,318     53.54%  WESTMORELAND COUNTY   
35    Donald J. Trump  Republican   5,192     55.61%        CLARKE COUNTY   
54    Donald J. Trump  Republican  21,245     71.07%      CAMPBELL COUNTY   
120   Donald J. Trump  Republican   8,365     84.10%           LEE COUNTY   
133   Donald J. Trump  Republican  13,294     60.66%        LOUISA COUNTY   
...               ...         ...     ...        ...                  ...   
2609  Donald J. Trump  Republican  25,106     57.50%      FAUQUIER COUNTY   
2639  Donald J. Trump  Republican   1,863     67.11%     BUENA VISTA CITY   
2726  Donald J. Trump  Republican  14,875     66.76%    GLOUCESTER COUNTY   
2756  Donald J. Trump  Republican  10,418     57.09%       HALIFAX COUNTY   
2846  Donald J. Trump  Republican   1,109     69.27%          NORTON CITY   

      percent  P_Democrat  P_Republican  
1      0.5354           0             1  
35     0.5561           0             1  
54     0.7107           0             1  
120    0.8410           0             1  
133    0.6066           0             1  
...       ...         ...           ...  
2609   0.5750           0             1  
2639   0.6711           0             1  
2726   0.6676           0             1  
2756   0.5709           0             1  
2846   0.6927           0             1  

[86 rows x 8 columns]

In [128]:
## Create dummy for Democratic majority in Senate election
vde2["S_Democrat"] = np.where(((vde2['Candidate'] == "Mark R. Warner") & (vde2['percent'] > 0.5)), 1, 0)

## Check
vde2.loc[vde2['S_Democrat'] == 1,]

Candidate Affiliation    Count Percentage               Locality  \
83    Mark R. Warner  Democratic  137,814     61.80%         LOUDOUN COUNTY   
150   Mark R. Warner  Democratic    3,889     58.16%     NORTHAMPTON COUNTY   
229   Mark R. Warner  Democratic   11,116     66.30%      HARRISONBURG CITY   
300   Mark R. Warner  Democratic   12,519     65.16%          DANVILLE CITY   
340   Mark R. Warner  Democratic    4,727     59.68%       BRUNSWICK COUNTY   
374   Mark R. Warner  Democratic    1,618     69.38%           EMPORIA CITY   
423   Mark R. Warner  Democratic    7,016     79.95%      FALLS CHURCH CITY   
469   Mark R. Warner  Democratic   65,071     79.37%        ALEXANDRIA CITY   
483   Mark R. Warner  Democratic    6,898     57.58%        WINCHESTER CITY   
539   Mark R. Warner  Democratic    9,179     68.88%           FAIRFAX CITY   
553   Mark R. Warner  Democratic  117,199     65.09%         HENRICO COUNTY   
604   Mark R. Warner  Democratic    7,289     56.72%          STAUNTON CITY   
686   Mark R. Warner  Democratic   25,685     52.07%      JAMES CITY COUNTY   
784   Mark R. Warner  Democratic    5,195     54.68%   PRINCE EDWARD COUNTY   
798   Mark R. Warner  Democratic    4,811     70.33%      WILLIAMSBURG CITY   
894   Mark R. Warner  Democratic   68,129     53.90%        CHESAPEAKE CITY   
1028  Mark R. Warner  Democratic   20,672     85.76%   CHARLOTTESVILLE CITY   
1129  Mark R. Warner  Democratic    2,700     61.29%    CHARLES CITY COUNTY   
1192  Mark R. Warner  Democratic    1,853     67.36%         LEXINGTON CITY   
1212  Mark R. Warner  Democratic   12,413     89.09%        PETERSBURG CITY   
1244  Mark R. Warner  Democratic   29,275     59.41%           SUFFOLK CITY   
1322  Mark R. Warner  Democratic  107,568     53.67%    CHESTERFIELD COUNTY   
1386  Mark R. Warner  Democratic   10,547     62.82%          MANASSAS CITY   
1415  Mark R. Warner  Democratic   42,730     67.31%       ALBEMARLE COUNTY   
1455  Mark R. Warner  Democratic    3,194     52.25%           ESSEX COUNTY   
1469  Mark R. Warner  Democratic    8,124     50.23%        CAROLINE COUNTY   
1512  Mark R. Warner  Democratic    4,673     50.75%          NELSON COUNTY   
1568  Mark R. Warner  Democratic    3,957     67.72%      MARTINSVILLE CITY   
1721  Mark R. Warner  Democratic   40,566     51.97%        STAFFORD COUNTY   
1767  Mark R. Warner  Democratic    2,924     57.62%          SUSSEX COUNTY   
1960  Mark R. Warner  Democratic   46,627     72.30%           HAMPTON CITY   
2043  Mark R. Warner  Democratic   91,222     83.27%          RICHMOND CITY   
2135  Mark R. Warner  Democratic   53,265     67.61%      NEWPORT NEWS CITY   
2204  Mark R. Warner  Democratic   66,152     74.01%           NORFOLK CITY   
2238  Mark R. Warner  Democratic   31,520     71.97%        PORTSMOUTH CITY   
2299  Mark R. Warner  Democratic    2,549     64.35%          FRANKLIN CITY   
2313  Mark R. Warner  Democratic   28,345     66.08%           ROANOKE CITY   
2371  Mark R. Warner  Democratic   18,587     51.85%         LYNCHBURG CITY   
2412  Mark R. Warner  Democratic   24,589     55.09%      MONTGOMERY COUNTY   
2425  Mark R. Warner  Democratic  120,753     53.91%    VIRGINIA BEACH CITY   
2560  Mark R. Warner  Democratic  144,162     64.24%  PRINCE WILLIAM COUNTY   
2584  Mark R. Warner  Democratic    2,439     55.43%           SURRY COUNTY   
2629  Mark R. Warner  Democratic    3,551     56.82%           RADFORD CITY   
2659  Mark R. Warner  Democratic    5,695     59.64%          HOPEWELL CITY   
2684  Mark R. Warner  Democratic    2,716     59.80%     GREENSVILLE COUNTY   
2698  Mark R. Warner  Democratic  102,880     79.37%       ARLINGTON COUNTY   
2745  Mark R. Warner  Democratic    8,495     67.48%    FREDERICKSBURG CITY   
2787  Mark R. Warner  Democratic  415,791     69.89%         FAIRFAX COUNTY   
2827  Mark R. Warner  Democratic    4,084     67.86%     MANASSAS PARK CITY   

      percent  P_Democrat  P_Republican  S_Democrat  
83    

In [129]:
## Create dummy for Republican majority in Senate election
vde2["S_Republican"] = np.where(((vde2['Candidate'] == "Daniel M. Gade") & (vde2['percent'] > 0.5)), 1, 0)

## Check
vde2.loc[vde2['S_Republican'] == 1,]

Candidate Affiliation   Count Percentage             Locality  \
5     Daniel M. Gade  Republican   5,009     50.94%  WESTMORELAND COUNTY   
39    Daniel M. Gade  Republican   5,214     56.22%        CLARKE COUNTY   
58    Daniel M. Gade  Republican  20,337     68.60%      CAMPBELL COUNTY   
124   Daniel M. Gade  Republican   7,600     77.16%           LEE COUNTY   
137   Daniel M. Gade  Republican  13,039     59.92%        LOUISA COUNTY   
...              ...         ...     ...        ...                  ...   
2613  Daniel M. Gade  Republican  24,937     58.75%      FAUQUIER COUNTY   
2643  Daniel M. Gade  Republican   1,724     62.76%     BUENA VISTA CITY   
2730  Daniel M. Gade  Republican  14,605     65.96%    GLOUCESTER COUNTY   
2760  Daniel M. Gade  Republican   9,824     54.31%       HALIFAX COUNTY   
2850  Daniel M. Gade  Republican     988     62.41%          NORTON CITY   

      percent  P_Democrat  P_Republican  S_Democrat  S_Republican  
5      0.5094           0             0           0             1  
39     0.5622           0             0           0             1  
58     0.6860           0             0           0             1  
124    0.7716           0             0           0             1  
137    0.5992           0             0           0             1  
...       ...         ...           ...         ...           ...  
2613   0.5875           0             0           0             1  
2643   0.6276           0             0           0             1  
2730   0.6596           0             0           0             1  
2760   0.5431           0             0           0             1  
2850   0.6241           0             0           0             1  

[84 rows x 10 columns]

In [130]:
## Export for review
vde2.to_csv("data/build/build2_vde.csv", index=False)

In [134]:
## Aggregate to county level for merge to 'Base'
locality = pd.pivot_table(vde2, index= ['Locality'], values= ['P_Democrat','P_Republican','S_Democrat','S_Republican'], aggfunc= np.sum)

locality.head()

P_Democrat  P_Republican  S_Democrat  S_Republican
Locality                                                            
ACCOMACK COUNTY            0             1           0             1
ALBEMARLE COUNTY           1             0           1             0
ALEXANDRIA CITY            1             0           1             0
ALLEGHANY COUNTY           0             1           0             1
AMELIA COUNTY              0             1           0             1

In [135]:
## Export for merge to base
locality.to_csv("data/build/build2_locality.csv")